# Building Custom Ansible Modules in Python

Have you ever wanted to type

`ansible -m my-really-cool-custom-module -a 'param1=foo param2=bar'`

... and have it actually work?

Then this section may be for you!

(much of this material is taken from Redhat's [Ansible Module Development Walkthrough][1])

[1]: http://docs.ansible.com/ansible/latest/dev_guide/developing_modules_general.html

*Don't* create a custom module until

- you've looked through all the built-in Ansible modules and can't find something that does what you want
- you've decided that using the `raw` and `script` modules is not powerful enough for what you need
- you know some Python

# Environment setup

Note: this becomes more useful when you want to do things like _test_ your module. 
For basic tasks, you should be fine just adding a `library=./modules` to your `ansible.cfg` and creating `.py` files there.

- Clone the Ansible repository: `$ git clone https://github.com/ansible/ansible.git`
- Change directory into the repository root dir: `$ cd ansible`
- Create a virtual environment: `$ python3 -m venv venv` (or for Python 2 $ `virtualenv venv`. Note, this requires you to install the virtualenv package: `$ pip install virtualenv`)
- Activate the virtual environment: `$ . venv/bin/activate`
- Install development requirements: `$ pip install -r requirements.txt`
- Run the environment setup script for each new dev shell process: `$ . hacking/env-setup`



In [9]:
cd ~/src/Classes/Ansible

/Users/rick446/src/Classes/Ansible


In [10]:
%%bash
git clone https://github.com/ansible/ansible.git
cd ansible
python3 -m venv venv
. venv/bin/activate
pip install -r requirements.txt
. hacking/env-setup

running egg_info
creating lib/ansible.egg-info
writing lib/ansible.egg-info/PKG-INFO
writing dependency_links to lib/ansible.egg-info/dependency_links.txt
writing requirements to lib/ansible.egg-info/requires.txt
writing top-level names to lib/ansible.egg-info/top_level.txt
writing manifest file 'lib/ansible.egg-info/SOURCES.txt'
reading manifest file 'lib/ansible.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lib/ansible.egg-info/SOURCES.txt'

Setting up Ansible to run out of checkout...

PATH=/Users/rick446/src/Classes/Ansible/ansible/bin:/Users/rick446/src/Classes/Ansible/ansible/test/runner:/Users/rick446/src/Classes/Ansible/ansible/venv/bin:/Users/rick446/.virtualenvs/ansible/bin:/Users/rick446/.virtualenvs/ansible/bin:/usr/local/heroku/bin:/usr/local/bin:/usr/local/share/python:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin:/opt/X11/bin:/Library/TeX/texbin:/Users/rick446/bin:/Users/rick446/bin
PYTHONPATH=/Users/rick446/src/Classes/Ansible/ansib

fatal: destination path 'ansible' already exists and is not an empty directory.
no previously-included directories found matching 'ticket_stubs'
no previously-included directories found matching 'hacking'


A minimal module can be found in `modules/simple_module.py` (a commented version is available in `modules/my_new_test_module.py`):

In [4]:
cat modules/simple_module.py

import os
from ansible.module_utils.basic import AnsibleModule

def run_module():
    module_args = dict(
        name=dict(type='str', required=True),
        new=dict(type='bool', required=False, default=False)
    )
    result = dict(
        changed=False,
        original_message='',
        message=''
    )
    module = AnsibleModule(
        argument_spec=module_args,
        supports_check_mode=True
    )
    if module.check_mode:
        return module.exit_json(**result)
    result['original_message'] = module.params['name']
    result['message'] = 'goodbye'
    result['num_files'] = len(os.listdir(os.environ['HOME']))
    if module.params['new']:
        result['changed'] = True
    if module.params['name'] == 'fail me':
        module.fail_json(msg='You requested this to fail', **result)
    module.exit_json(**result)


if __name__ == '__main__':
    run_module()


In [5]:
%%bash
ansible localhost -m simple_module

localhost | FAILED! => {
    "changed": false,
    "msg": "missing required arguments: name"
}


In [6]:
%%bash
ansible localhost -m simple_module -a name=foo

localhost | SUCCESS => {
    "changed": false,
    "message": "goodbye",
    "num_files": 151,
    "original_message": "foo"
}


In [7]:
%%bash
ansible localhost -m simple_module -a 'name="fail me"'

localhost | FAILED! => {
    "changed": false,
    "message": "goodbye",
    "msg": "You requested this to fail",
    "num_files": 151,
    "original_message": "fail me"
}


In [8]:
%%bash
ansible localhost -m simple_module -a 'name=foo new=true'

localhost | SUCCESS => {
    "changed": true,
    "message": "goodbye",
    "num_files": 151,
    "original_message": "foo"
}


## Lab: Modify your module

~~Modify the module to report on the number of files in the user's home directory (hint: use `os.listdir()` and `os.environ['HOME']`. Confirm that it works locally.~~

Modify the module `simple_module` to return the current logged-in user's username

(you'll need to update `result` to add a `username=os.environ['USER']` argument)

Test it with both Vagrant and remotely (use -i inventory with `ansible`)